In [1]:
!mamba install pandas==1.3.3 -y
!mamba install requests==2.26.0 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.27.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pandas==1.3.3']

[+] 0.0s
opt/ibm/custom-channels/meta-wscloud/linux-64      129.0 B @   1.5MB/s 

In [2]:
import glob
import pandas as pd
from datetime import datetime

In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-09-21 17:04:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.3’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-09-21 17:04:31 (21.9 MB/s) - ‘bank_market_cap_1.json.3’ saved [2815/2815]

--2023-09-21 17:04:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%2

In [4]:
# Write your code here
dfExchRate = pd.read_csv("exchange_rates.csv", index_col = 0) 
exchange_rate = dfExchRate.loc["GBP","Rates"]
print(dfExchRate.head())
print(exchange_rate)

        Rates
AUD  1.297088
BGN  1.608653
BRL  5.409196
CAD  1.271426
CHF  0.886083
0.7323984208000001


In [38]:
logfile = "bank_market_cap_logfile.txt"
#log function
def log(message):
    timestamp_format = "%Y-%h-%d-%H:%M:%S" #year-month-date-Hour:Minute:Second
    time = datetime.now() #current time
    timestamp = time.strftime(timestamp_format)
    with open(logfile, "a") as f:
        f.write(timestamp + ',' + message + '\n')

In [39]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [40]:
columns=['Name','Market Cap (US$ Billion)']

In [41]:
def extract():
    # Write your code here
    dataframe = extract_from_json("bank_market_cap_1.json")[columns]
    return dataframe
    

In [42]:
log("ETL Job Started")
log("Extract phase started")
extract_data=extract()
print(extract_data.head(5))
log("Extract phase ended")

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399


In [43]:
def transform(df, rate):
    # Write your code here
    df['Market Cap (US$ Billion)'] = df['Market Cap (US$ Billion)']*rate
    df = df.rename(columns = {'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'}, inplace=False)
    return df
    

In [44]:
log("Transform phase started")
transform_data = transform(extract_data, exchange_rate)
print(transform_data.head(5))
log("Transform phase ended")

                                      Name  Market Cap (GBP$ Billion)
0                           JPMorgan Chase                 286.319444
1  Industrial and Commercial Bank of China                 252.834188
2                          Bank of America                 238.271911
3                              Wells Fargo                 225.588235
4                  China Construction Bank                 188.518621


In [45]:
targetfile = "bank_market_gbp.csv"
def load(targetfile, data):
    data.to_csv(targetfile, index=False)

In [46]:
log("Load phase started")
load(targetfile, transform_data)
log("Load phase ended")